In [1]:
import pandas as pd
import numpy as np

Load the necessary datafiles for merge

In [2]:
#specify where to read raw datafiles and where to save processed files
filepath_raw = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw CBS data/'
filepath_processed = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Processed CBS data/'
filepath_other = 'C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw data other/'

#BAG register
sourceBAG = 'https://data.nlextract.nl/bag/csv/'
df = pd.read_csv(filepath_raw + 'bagadres-full_2019.csv', delimiter = ';')#, low_memory = False)

#Rijksmonumenten
source = '' 
rijksmonumenten = pd.read_csv(filepath_other + 'rijksmonumenten.csv', encoding = 'unicode_escape')

#Provisional Energy labels
source_rvo = 'https://www.rvo.nl/onderwerpen/duurzaam-ondernemen/gebouwen/hulpmiddelen-tools-en-inspiratie-gebouwen/ep-online'
labels_1 = pd.read_csv(filepath_raw + 'labels 1.csv', delimiter = ',')#, low_memory = False)
labels_2 = pd.read_csv(filepath_raw + 'labels 2.csv', delimiter = ',')#, low_memory = False)
labels_3 = pd.read_csv(filepath_raw + 'labels 3.csv', delimiter = ',')#, low_memory = False)
labels_4 = pd.read_csv(filepath_raw + 'labels 4.csv', delimiter = ',')#, low_memory = False)
labels_5 = pd.read_csv(filepath_raw + 'labels 5.csv', delimiter = ',')#, low_memory = False)
labels_6 = pd.read_csv(filepath_raw + 'labels 6.csv', delimiter = ',')#, low_memory = False)
labels_7 = pd.read_csv(filepath_raw + 'labels 7.csv', delimiter = ',')#, low_memory = False)
labels_8 = pd.read_csv(filepath_raw + 'labels 8.csv', delimiter = ',')#, low_memory = False)

#Natural gas use
sourceGAS = 'https://opendata.cbs.nl/statline/#/CBS/nl/dataset/83878ned/table?ts=1580216940954'
gas_levering = pd.read_csv(filepath_raw + 'Aardgaslevering_woningkenmerken.csv', delimiter = ';')
meta_data_gas = pd.read_csv(filepath_raw + 'Meta_data_woningkenmerken.csv', delimiter = ',')
meta_dict = meta_data_gas[['Key','Title']].set_index(['Key']).to_dict()['Title']

#Electricity use
sourceELEK = 'https://opendata.cbs.nl/statline/#/CBS/nl/dataset/81528NED/table?fromstatweb'
elec_use = pd.read_csv(filepath_raw + 'Elektriciteitsgebruik per woningtype per samenstelling per type.csv', delimiter = ';')
elec_use = elec_use.replace(meta_dict)[['Woningkenmerken','GebruiksOppervlakteklasse','BewonersklasseWoningen','ElektriciteitsleveringenOpenbareNet_1']]

#Municipalities 2019
Mun_code = pd.read_csv(filepath_raw + 'Gemeenten alfabetisch 2019.csv', delimiter = ',',low_memory=False,index_col = 0)
Mun_code_dict = Mun_code[['GemeentecodeGM','Gemeentenaam']].set_index(['GemeentecodeGM']).to_dict()['Gemeentenaam']

#Households 2019
source_households = ''
gem_huishouden = pd.read_csv(filepath_raw + 'postcodes_huishoudengrootte.csv', delimiter = ';')

#Heating grids
source_stadswarmte = 'https://opendata.cbs.nl/statline/#/CBS/nl/dataset/81528NED/table?fromstatweb'
stadswarmte = pd.read_csv(filepath_raw + 'Stadsverwarming_particuliere_woningen.csv', delimiter = ';')
stadswarmte = stadswarmte.replace(Mun_code_dict).replace(meta_dict)[['RegioS','Stadsverwarming_3']].rename(columns = {'RegioS':'gemeente','Stadsverwarming_3':'Stadsverwarming'})

warmtenetten_CEDELFT = pd.read_csv('C:/Users/Dirk/Google Drive/Dirk/TU/Afstuderen/Afstuderen/Thesis/3. Formalisation/Raw data other/warmtenetten.csv',header = 2)

C:\Users\Dirk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Needed to prevent errors in Vensim and Netlogo, which don't deal well with non-latins
df.gemeente = df.gemeente.replace({"Nuenen, Gerwen en Nederwetten": "Nuenen Gerwen en Nederwetten", "Noardeast-Fryslân": "Noardeast-Fryslan" , "Súdwest-Fryslân":"Sudwest-Fryslan" })

In [4]:
#openbareruimte;huisnummer;huisletter;huisnummertoevoeging;postcode;woonplaats;gemeente;provincie;nummeraanduiding;verblijfsobjectgebruiksdoel;oppervlakteverblijfsobject;verblijfsobjectstatus;object_id;object_type;nevenadres;pandid;pandstatus;pandbouwjaar;x;y;lon;lat
df_woonfunctie = df[df.verblijfsobjectgebruiksdoel.str.contains("woonfunctie") == True][['gemeente','postcode','huisnummer','huisletter','huisnummertoevoeging','oppervlakteverblijfsobject','pandbouwjaar','openbareruimte','pandstatus']]
df_woonfunctie = df_woonfunctie.query('pandstatus == "Pand in gebruik" | pandstatus == "Pand in gebruik (niet ingemeten)"')
df_woonfunctie['huisletter'] = df_woonfunctie['huisletter'].str.upper()
df_woonfunctie['huisnummer'] = pd.to_numeric(df_woonfunctie['huisnummer'])

df_woonfunctie.head()

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus
1,Amsterdam,1011AB,105,NaN,2,72,1884.0,De Ruijterkade,Pand in gebruik
2,Amsterdam,1011AB,105,NaN,3,84,1884.0,De Ruijterkade,Pand in gebruik
5,Amsterdam,1011AB,106,NaN,2,82,1884.0,De Ruijterkade,Pand in gebruik
6,Amsterdam,1011AB,106,NaN,3,60,1884.0,De Ruijterkade,Pand in gebruik
8,Amsterdam,1011AB,107,NaN,5,308,1005.0,De Ruijterkade,Pand in gebruik


In [5]:
#Merge all separate files, rename columns, rearrange types
labels_total = pd.concat([labels_1,labels_2,labels_3,labels_4,labels_5,labels_6,labels_7,labels_8])
labels_total = labels_total.rename(columns = {'POSTCODE_WONING':'postcode', 'HUISNUMMER_WONING':'huisnummer','BOUWJAAR_WONING':'pandbouwjaar'})
labels_total = labels_total.reset_index().drop('index', axis = 1)
labels_total['huisnummer'] = pd.to_numeric(labels_total['huisnummer'])
labels_total['HUISNUMMER_TOEV_NUMMER'] = pd.to_numeric(labels_total['HUISNUMMER_TOEV_WONING'], errors = 'coerce')
labels_total['huisletter'] = labels_total['HUISNUMMER_TOEV_WONING'][labels_total['HUISNUMMER_TOEV_NUMMER'].isnull()].str.upper()
labels_total['huisnummertoevoeging'] = labels_total['HUISNUMMER_TOEV_NUMMER']
labels_total['huisnummertoevoeging'] = [x if len(str(x)) > 1 else np.nan for x in labels_total['HUISNUMMER_TOEV_WONING']] 
labels_total = labels_total.drop(['HUISNUMMER_TOEV_WONING', 'HUISNUMMER_TOEV_NUMMER' ], axis = 1)

#Result
labels_total.head()

,postcode,huisnummer,pandbouwjaar,WONING_TYPE,VOORL_BEREKEND,huisletter,huisnummertoevoeging
0,9901KB,16,2002,C5,B,NaN,NaN
1,9901KB,20,2002,C5,B,NaN,NaN
2,9901AD,15,1925,C1,G,NaN,NaN
3,9901AD,6,1950,C5,E,NaN,NaN
4,9901AD,10,1930,C5,G,A,NaN


In [6]:
#Group dataframes, sort by postcode, backwards fill so that Label Entries will be filled in for BAG register. Drop duplicates.
df_total = df_woonfunctie.merge(labels_total, on = ['postcode','huisnummer','huisletter','huisnummertoevoeging','pandbouwjaar'], how = 'outer')
df_total = df_total.sort_values(by = ['postcode', 'huisnummer', 'huisletter', 'huisnummertoevoeging', 'pandbouwjaar'], axis = 0).reset_index().drop('index', axis = 1)
df_total['WONING_TYPE'] = df_total.WONING_TYPE.bfill()
df_total['VOORL_BEREKEND'] = df_total.VOORL_BEREKEND.bfill()
df_total = df_total.dropna(subset = ['gemeente']).rename(columns = {'WONING_type':'woningtype', 'VOORL_BEREKEND':'Energielabel'})
df_total = df_total.reset_index().drop('index', axis = 1)
df_total['WONING_TYPE'] = df_total.WONING_TYPE.replace(meta_dict)
df_total = df_total.rename(columns = {'WONING_TYPE':'Woningkenmerken'})

#Result
df_total.head()
#FINISHED DATA CLEANING MERGE BAG AND LABEL REGISTER

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus,Woningkenmerken,Energielabel
0,Amsterdam,1011AB,105,NaN,2,72.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G
1,Amsterdam,1011AB,105,NaN,3,84.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G
2,Amsterdam,1011AB,106,NaN,2,82.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G
3,Amsterdam,1011AB,106,NaN,3,60.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G
4,Amsterdam,1011AB,107,NaN,5,308.0,1005.0,De Ruijterkade,Pand in gebruik,Appartement,G


In [7]:
rijksmonumenten = rijksmonumenten [['RIJKSMONNR', 'CBSCATEGOR','POSTCODE','HUISNUMMER', 'TOEVOEGING']].set_index('CBSCATEGOR').loc['Gebouwen, woonhuizen'].reset_index()

In [8]:
rijksmonumenten = rijksmonumenten.rename(columns = {"POSTCODE":"postcode","HUISNUMMER":'huisnummer','TOEVOEGING':'huisletter'})
rijksmonumenten.postcode = rijksmonumenten.postcode.str.replace(" ", "")

In [9]:
df_total = df_total.merge(rijksmonumenten, on = ['postcode', 'huisnummer','huisletter'], how = 'left')

In [10]:
#Create age groups and classify all entries
df_total['leeftijd'] = 2020 - df_total['pandbouwjaar']
group_dict = {}
x = 9
for i in range (2020):
    if i>x:
        x+=10
    y = x - 9
    if i < 100:
        group_dict[i] = 'Age' + str(y) + 'to' + str(x)
    if i > 99:
        group_dict[i] = 'Age100plus'
df_total['Age group'] = df_total.leeftijd.map(group_dict)

In [11]:
#Count entries per age group, sort so that vensim will understand and fill NaN's with 0's for non existing age groups in municipalities
init_stock_age_municipality = df_total.groupby(by = ['gemeente', 'Age group'], sort = False).count()[['openbareruimte']]
init_stock_age_municipality = init_stock_age_municipality.reset_index().pivot(index='gemeente', columns='Age group', values='openbareruimte')
init_stock_age_municipality = init_stock_age_municipality[['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']].fillna(0)

#End result count per age group
init_stock_age_municipality.head()

Age group,Age0to9,Age10to19,Age20to29,Age30to39,Age40to49,Age50to59,Age60to69,Age70to79,Age80to89,Age90to99,Age100plus
gemeente,,,,,,,,,,,
's-Gravenhage,12740.0,29379.0,26685.0,23337.0,15754.0,16366.0,29972.0,5230.0,26431.0,34646.0,45090.0
's-Hertogenbosch,5617.0,8503.0,8411.0,10448.0,12809.0,11293.0,6448.0,1711.0,2635.0,1542.0,3939.0
Aa en Hunze,230.0,888.0,1255.0,1560.0,1982.0,1758.0,841.0,334.0,559.0,557.0,1247.0
Aalsmeer,926.0,3618.0,1047.0,1853.0,1163.0,1892.0,1030.0,414.0,622.0,404.0,259.0
Aalten,556.0,924.0,1243.0,1681.0,2105.0,1585.0,1053.0,395.0,672.0,560.0,1239.0


In [12]:
#Translate names based on meta_dict and rename. Save as dictionary so that information can be loaded into df_total. 
gas_levering = gas_levering.replace(meta_dict).rename(columns = {'AardgasleveringenOpenbareNet_1':'Gaslevering gemiddeld m3.m2'}).drop( ['ID','Percentielen','Perioden'],axis = 1).replace({'1000 tot 1946': "Voor 1946"})
gas_levering['Energielabelklasse'] = gas_levering.Energielabelklasse.str.replace('-label','')
gas_levering = gas_levering.set_index(['Woningkenmerken','GebruiksOppervlakteklasse','Bouwjaarklasse','Energielabelklasse'])
gas_levering = pd.to_numeric(gas_levering['Gaslevering gemiddeld m3.m2'], errors = 'coerce').ffill()
levering_dict = gas_levering.to_dict()

In [13]:
#Add GebruiksOppervlakteklasse
group_dict = {}
x = 50
y = 0
for i in range (100000):
    if i <50:
        y = 15
    if i <100 and i >49:
        if i==x:
            x+=25
        y = x - 25 
    if i <150 and i > 99:
        if i==x:
            x+=50
        y = x - 50
    if i <250 and i > 149:
        if i==x:
            x+=100
        y = x - 100
    if i <500 and i > 249:
        if i==x:
            x+=250
        y = x - 250
    group_dict[i] = str(y) + ' tot ' + str(x) + ' m2'
else: 
    group_dict[i] = 'Groter dan 500 m2'
    
df_total['GebruiksOppervlakteklasse'] = df_total.oppervlakteverblijfsobject.map(group_dict)

In [14]:
# Add Bouwjaarklasse
group_dict = {}
for i in range (3000):
    if i <1946:
        x = ''
        y = 'Voor 1946'
    if i <1965 and i >1945:
        x = '1946 tot '
        y = '1965'
    if i <1975 and i > 1964:
        x = '1965 tot '
        y = '1975'
    if i <1992 and i > 1974:
        x = '1975 tot '
        y = '1992'
    if i <2000 and i > 1991:
        x = '1992 tot '
        y = '2000'
    if i <2014 and i > 1999:
        x = '2000 tot '
        y = '2014'
    if i >2013:
        x = ''
        y = 'Vanaf 2014'
    group_dict[i] = x + y
    
df_total['Bouwjaarklasse'] = df_total.pandbouwjaar.map(group_dict)

In [15]:
#Add gas use per m2 based on Woningkenmerken, gebruiksoppervlakteklasse, Bouwjaarklasse & Energy Label 
df_total['gasverbruik m3/m2'] = pd.Series(list(zip(df_total.Woningkenmerken, df_total.GebruiksOppervlakteklasse, df_total.Bouwjaarklasse, df_total.Energielabel))).map(levering_dict)
df_total.head()

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus,Woningkenmerken,Energielabel,CBSCATEGOR,RIJKSMONNR,leeftijd,Age group,GebruiksOppervlakteklasse,Bouwjaarklasse,gasverbruik m3/m2
0,Amsterdam,1011AB,105,NaN,2,72.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2
1,Amsterdam,1011AB,105,NaN,3,84.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4
2,Amsterdam,1011AB,106,NaN,2,82.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4
3,Amsterdam,1011AB,106,NaN,3,60.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2
4,Amsterdam,1011AB,107,NaN,5,308.0,1005.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,1015.0,Age100plus,250 tot 500 m2,Voor 1946,10.6


In [16]:
#Data for which no estimate can be calculated? Deemed acceptable
df_total[df_total['gasverbruik m3/m2'].isnull()].groupby(by = 'Woningkenmerken').count()['gemeente']

Woningkenmerken
2-onder-1-kapwoning       2
Appartement               2
C0                     4555
N                       434
O                      6571
Tussenwoning              1
Vrijstaande woning        6
Name: gemeente, dtype: int64

In [17]:
#Estimate yearly gasuse
df_total['geschat gasverbruik jaarlijks'] = df_total['gasverbruik m3/m2'] * df_total['oppervlakteverblijfsobject']

#Result
df_total.head()

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus,Woningkenmerken,Energielabel,CBSCATEGOR,RIJKSMONNR,leeftijd,Age group,GebruiksOppervlakteklasse,Bouwjaarklasse,gasverbruik m3/m2,geschat gasverbruik jaarlijks
0,Amsterdam,1011AB,105,NaN,2,72.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,1166.4
1,Amsterdam,1011AB,105,NaN,3,84.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1209.6
2,Amsterdam,1011AB,106,NaN,2,82.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1180.8
3,Amsterdam,1011AB,106,NaN,3,60.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,972.0
4,Amsterdam,1011AB,107,NaN,5,308.0,1005.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,1015.0,Age100plus,250 tot 500 m2,Voor 1946,10.6,3264.8


In [18]:
# Mean gas use per municipality and age group
mean_agg_gas_use = df_total.groupby(by = ['gemeente', 'Age group'], sort = False).mean()[['geschat gasverbruik jaarlijks']]
mean_agg_gas_use = mean_agg_gas_use.reset_index().pivot(index='gemeente', columns='Age group', values='geschat gasverbruik jaarlijks')
mean_agg_gas_use = mean_agg_gas_use[['Age0to9','Age10to19','Age20to29','Age30to39','Age40to49','Age50to59','Age60to69','Age70to79','Age80to89','Age90to99','Age100plus']].ffill(axis = 1)

#Result
mean_agg_gas_use.head()

Age group,Age0to9,Age10to19,Age20to29,Age30to39,Age40to49,Age50to59,Age60to69,Age70to79,Age80to89,Age90to99,Age100plus
gemeente,,,,,,,,,,,
's-Gravenhage,744.187097,954.202890,932.552946,1167.304593,1016.171377,1290.189788,1081.408656,1139.194491,1343.505374,1333.193456,1436.077738
's-Hertogenbosch,878.479469,1046.408974,1219.332235,1209.071197,1430.351761,1526.356947,1427.559820,1497.175628,1638.161825,1598.853931,1649.532644
Aa en Hunze,1407.026957,1627.895065,1685.798008,1633.926346,1850.442129,1974.310637,2183.239286,2492.975150,2818.279070,3152.179354,3423.752899
Aalsmeer,1088.791469,1177.167791,1154.103349,1278.854428,1567.878934,1522.299204,1563.845728,1532.150725,1853.889694,1854.456468,1872.065251
Aalten,1090.958559,1175.891342,1352.804183,1455.255741,1601.684078,1725.373060,1833.407502,1900.774365,2321.347690,2418.624464,2646.939176


In [19]:
#Average size households per postal code to calculate electricity use per household
gem_huishouden = gem_huishouden[['Postcode','GemiddeldeHuishoudensgrootte_2']].rename(columns = {'GemiddeldeHuishoudensgrootte_2':'Gemiddelde grootte huishouden'}).drop(0)
gem_huishouden['Postcode'] = gem_huishouden.Postcode.str.strip('PC')
gem_huishouden['Postcode'] = pd.to_numeric(gem_huishouden.Postcode)
gem_huishouden['Gemiddelde grootte huishouden'] = pd.to_numeric(gem_huishouden['Gemiddelde grootte huishouden'])

#Result
gem_huishouden.head()

,Postcode,Gemiddelde grootte huishouden
1,999,1.37
2,1011,1.49
3,1012,1.41
4,1013,1.66
5,1014,1.92


In [20]:
#Add column with only numbers of postal code to merge with household size
df_total['Postcode'] = df_total.postcode.map(lambda x: int(str(x)[:4]))
df_total = df_total.merge(gem_huishouden, how = 'left').drop('Postcode',axis = 1)

#Result
df_total.head()

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus,Woningkenmerken,Energielabel,CBSCATEGOR,RIJKSMONNR,leeftijd,Age group,GebruiksOppervlakteklasse,Bouwjaarklasse,gasverbruik m3/m2,geschat gasverbruik jaarlijks,Gemiddelde grootte huishouden
0,Amsterdam,1011AB,105,NaN,2,72.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,1166.4,1.49
1,Amsterdam,1011AB,105,NaN,3,84.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,"Gebouwen, woonhuizen",518411.0,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1209.6,1.49
2,Amsterdam,1011AB,106,NaN,2,82.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1180.8,1.49
3,Amsterdam,1011AB,106,NaN,3,60.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,972.0,1.49
4,Amsterdam,1011AB,107,NaN,5,308.0,1005.0,De Ruijterkade,Pand in gebruik,Appartement,G,NaN,NaN,1015.0,Age100plus,250 tot 500 m2,Voor 1946,10.6,3264.8,1.49


In [21]:
elec_use['BewonersklasseWoningen'] = elec_use.BewonersklasseWoningen.map(lambda x: int(str(x)[:1]))
elec_use = elec_use[['Woningkenmerken','GebruiksOppervlakteklasse','BewonersklasseWoningen','ElektriciteitsleveringenOpenbareNet_1']]
elec_use['ElektriciteitsleveringenOpenbareNet_1'] = pd.to_numeric(elec_use.ElektriciteitsleveringenOpenbareNet_1, errors = 'coerce')

#Result
elec_use.head()

,Woningkenmerken,GebruiksOppervlakteklasse,BewonersklasseWoningen,ElektriciteitsleveringenOpenbareNet_1
0,Appartement,15 tot 50 m2,1,1366.4
1,Appartement,15 tot 50 m2,2,872.0
2,Appartement,15 tot 50 m2,3,654.1
3,Appartement,15 tot 50 m2,4,551.4
4,Appartement,15 tot 50 m2,5,426.6


In [22]:
# This is probably not the best way to do it, but does the trick
# Add intermediate values for size of households to get better estimation of electricity use
x = elec_use.Woningkenmerken.unique()
y = elec_use.GebruiksOppervlakteklasse.unique()
z = elec_use.BewonersklasseWoningen.unique()
for i in x:
    for j in y:
        for k in z:
            for a in range (1,10):
                elec_use = elec_use.append({'Woningkenmerken': i, 'GebruiksOppervlakteklasse': j, 'BewonersklasseWoningen': k + a/10 }, ignore_index=True)

In [23]:
# Create dictionary of electricity use based on characteristics
elec_use = elec_use.sort_values(by = ['Woningkenmerken','GebruiksOppervlakteklasse','BewonersklasseWoningen']).interpolate().rename(columns = {'BewonersklasseWoningen':'Gemiddelde grootte huishouden','ElektriciteitsleveringenOpenbareNet_1':'Elektriciteitsverbruik pp'})
elec_dict = elec_use.set_index(['Woningkenmerken','GebruiksOppervlakteklasse','Gemiddelde grootte huishouden']).to_dict()

In [24]:
#Round household size to fit with dictionary.
df_total['Gemiddelde grootte huishouden'] = df_total['Gemiddelde grootte huishouden'].round(1)
df_total['elektriciteits verbruik pp'] = pd.Series(list(zip(df_total.Woningkenmerken, df_total.GebruiksOppervlakteklasse, df_total['Gemiddelde grootte huishouden']))).map(elec_dict['Elektriciteitsverbruik pp'])
df_total['geschat elektriciteitsverbruik jaarlijks'] = df_total['elektriciteits verbruik pp'] * df_total['Gemiddelde grootte huishouden']

#Result
df_total.head()

,gemeente,postcode,huisnummer,huisletter,huisnummertoevoeging,oppervlakteverblijfsobject,pandbouwjaar,openbareruimte,pandstatus,Woningkenmerken,...,RIJKSMONNR,leeftijd,Age group,GebruiksOppervlakteklasse,Bouwjaarklasse,gasverbruik m3/m2,geschat gasverbruik jaarlijks,Gemiddelde grootte huishouden,elektriciteits verbruik pp,geschat elektriciteitsverbruik jaarlijks
0,Amsterdam,1011AB,105,NaN,2,72.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,...,518411.0,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,1166.4,1.5,1266.00,1899.000
1,Amsterdam,1011AB,105,NaN,3,84.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,...,518411.0,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1209.6,1.5,1433.75,2150.625
2,Amsterdam,1011AB,106,NaN,2,82.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,...,NaN,136.0,Age100plus,75 tot 100 m2,Voor 1946,14.4,1180.8,1.5,1433.75,2150.625
3,Amsterdam,1011AB,106,NaN,3,60.0,1884.0,De Ruijterkade,Pand in gebruik,Appartement,...,NaN,136.0,Age100plus,50 tot 75 m2,Voor 1946,16.2,972.0,1.5,1266.00,1899.000
4,Amsterdam,1011AB,107,NaN,5,308.0,1005.0,De Ruijterkade,Pand in gebruik,Appartement,...,NaN,1015.0,Age100plus,250 tot 500 m2,Voor 1946,10.6,3264.8,1.5,2432.75,3649.125


In [25]:
stadswarmte['gemeente'] = stadswarmte.gemeente[stadswarmte.gemeente.str.contains("GM") == False]
stadswarmte = stadswarmte.dropna()

gemeentes = pd.DataFrame()
gemeentes['gemeente'] = Mun_code.reset_index().Gemeentenaam.unique()
stadswarmte = stadswarmte.reset_index().merge(gemeentes,how = 'outer').set_index('gemeente').sort_index().drop('index', axis = 1)
stadswarmte['Stadsverwarming'] = pd.to_numeric(stadswarmte.Stadsverwarming, errors = 'coerce').fillna(0)
stadswarmte['Stadsverwarming'] = stadswarmte['Stadsverwarming'] /100
stadswarmte['Totaal woningen'] =(df_total.groupby(by = 'gemeente').count()['openbareruimte'])
stadswarmte['Woningen stadsverwarming'] = (stadswarmte.Stadsverwarming * stadswarmte['Totaal woningen']).round()

#Result 
stadswarmte.head()

,Stadsverwarming,Totaal woningen,Woningen stadsverwarming
gemeente,,,
's-Gravenhage,0.088,265630,23375.0
's-Hertogenbosch,0.000,73356,0.0
Aa en Hunze,0.000,11211,0.0
Aalsmeer,0.000,13228,0.0
Aalten,0.000,12013,0.0


In [26]:
#Save
stadswarmte.to_csv(filepath_processed + 'city heating.csv')

# Save total dataset here to reload in other files

In [27]:
#SAVE TOTAL DATASET
df_total.to_csv(filepath_processed + 'Total_dataset.csv', encoding = 'utf-8')

In [28]:
df_woonfunctie.gemeente.unique()

array(['Amsterdam', 'Diemen', 'Ouder-Amstel', 'Landsmeer',
       'Edam-Volendam', 'Waterland', 'Haarlemmermeer', 'Amstelveen',
       'Hilversum', 'Wijdemeren', 'Laren', 'Blaricum', 'Huizen', 'Almere',
       'Weesp', 'De Ronde Venen', 'Stichtse Vecht', 'Gooise Meren',
       'Uithoorn', 'Nieuwkoop', 'Aalsmeer', 'Purmerend', 'Wormerland',
       'Beemster', 'Alkmaar', 'Castricum', 'Zaanstad', 'Oostzaan',
       'Enkhuizen', 'Drechterland', 'Stede Broec', 'Medemblik', 'Hoorn',
       'Koggenland', 'Opmeer', 'Heerhugowaard', 'Langedijk', 'Schagen',
       'Hollands Kroon', 'Den Helder', 'Texel', 'Heiloo', 'Bergen (NH.)',
       'Uitgeest', 'Beverwijk', 'Velsen', 'Heemskerk', 'Haarlem',
       'Zandvoort', 'Bloemendaal', 'Heemstede', 'Kaag en Braassem',
       'Lisse', 'Teylingen', 'Hillegom', 'Noordwijk', 'Katwijk',
       'Wassenaar', 'Voorschoten', 'Leidschendam-Voorburg', 'Rijswijk',
       'Westland', 'Leiden', 'Oegstgeest', 'Leiderdorp', 'Zoeterwoude',
       'Alphen aan den Rijn',